In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-12-15"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
29208,2024-12-15,Macedônia Do Norte Prva Liga,15:00,Cair 2030,MKK Kumanovo,1.19,4.36,162.5,1.83,1.83,-11.5,2.00,1.72,S0VCxzA2,0.840336,0.229358,0.546448,0.546448,0.069694,199.326,167.125077,0.838451,0.6,1.341641,2.236068,116.07,2.490,1.605506,0.644782,-38.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,73,74,1.59,3.19,217.098,0.000,0.807758,0.000000,0.106446,2.95,0.590,0.322034,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
29209,2024-12-15,Noruega Blno,12:00,Fyllingen,Centrum Tigers,1.03,8.30,169.5,1.80,1.86,-17.5,1.98,1.70,bZz55XuJ,0.970874,0.120482,0.555556,0.537634,0.091356,1038.264,2074.944892,1.998475,3.0,0.000000,0.000000,108.12,8.500,16.491369,1.940161,166.0,142.700,51.326805,0.359683,1.2,1.643168,1.369306,130.53,2.292,1.561320,0.681204,-50.0,106,57,1.02,2.29,107.506,128.594,1.101965,0.023184,0.107603,0.56,0.112,0.267857,0.826163,0.9,0.073837,-1.64,-0.328,-22.256098,0.596226,0.6,0.003774
29210,2024-12-15,Noruega Blno,12:00,Kongsberg Miners,Bærum,1.20,3.84,163.5,1.82,1.84,-10.5,1.94,1.73,xKnd7gB6,0.833333,0.260417,0.549451,0.543478,0.093750,108.138,24.981719,0.231017,2.4,1.341641,0.559017,149.35,1.154,0.168018,0.145596,113.0,212.226,105.788951,0.498473,0.0,0.000000,NaN,165.48,3.056,1.545147,0.505611,-89.0,103,84,1.45,1.97,101.916,209.036,0.740779,0.007728,0.080922,-1.71,-0.342,-0.584795,0.846191,0.7,-0.146191,-3.65,-0.730,-3.890411,0.332326,0.4,0.067674
29211,2024-12-15,Países Baixos Wbl Feminina,10:00,Den Helder F,QSTA United F,1.31,3.05,130.5,1.88,1.79,-8.5,2.01,1.68,dYSXjksh,0.763359,0.327869,0.531915,0.558659,0.091228,150.874,95.659043,0.634033,0.6,1.341641,2.236068,66.12,2.466,1.559673,0.632471,-20.0,325.950,267.608207,0.821010,1.8,1.643168,0.912871,466.40,4.878,3.986185,0.817176,-12.0,57,53,1.16,8.80,0.000,0.000,0.564388,0.034681,0.126474,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
29212,2024-12-15,Portugal Lpb,11:15,Vitória SC,CD Póvoa,1.52,2.30,151.5,1.80,1.86,-5.5,2.05,1.66,fFoRBEm2,0.657895,0.434783,0.555556,0.537634,0.092677,158.644,60.171009,0.379283,1.8,1.643168,0.912871,137.74,2.010,0.802932,0.399469,1.0,238.530,205.021167,0.859519,0.6,1.341641,2.236068,202.62,3.508,3.333815,0.950346,-19.0,71,66,1.94,3.07,171.750,279.652,0.288766,0.023184,0.148664,-2.26,-0.452,-1.150442,0.600393,0.5,-0.100393,-2.79,-0.558,-2.329749,0.315729,0.4,0.084271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29452,2024-12-15,Eua Ncaa,19:00,CS Fullerton,Denver,1.69,2.18,0.0,0.00,0.00,-2.5,1.80,1.91,GK22hTo1,0.591716,0.458716,inf,inf,0.050432,313.924,332.884839,1.060399,0.6,1.341641,2.236068,284.92,5.894,7.395511,1.254752,-57.0,202.108,87.704761,0.433950,1.2,1.643168,1.369306,301.92,2.866,1.135707,0.396269,-32.0,68,68,4.19,4.44,140.758,246.754,0.179061,NaN,0.041931,-1.40,-0.280,-2.464286,0.570823,0.4,-0.170823,-0.56,-0.112,-10.535714,0.302031,0.3,-0.002031
29453,2024-12-15,Eua Ncaa,19:00,Illinois State,St. Louis Billikens,2.10,1.73,0.0,0.00,0.00,2.5,1.80,1.91,WjJBPyin,0.476190,0.578035,inf,inf,0.054225,215.420,101.899727,0.473028,1.2,1.643168,1.369306,

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
29222,15:00,Portugal Proliga,Benfica B,Barreirense,1.68,Back Home
29240,14:00,Itália Série A2,Cento,Nardo,1.59,Back Home
29257,04:05,Japão B.League,Brave Thunders,Hokkaido,2.26,Back Home
29261,02:00,Japão B2.League,Kumamoto,Aomori,2.12,Back Home
29281,17:00,Eua Ncaa,Texas,Arkansas-Pine Bluff,1.72,Back Home
29322,09:00,Turquia Tb2L,Tesvikiye,Gemlik,1.95,Back Home
29369,13:00,Espanha Acb,Joventut,Tenerife,1.89,Back Home
29382,14:15,Itália Liga A,Trapani,Trento,1.58,Back Home
29402,12:00,Europa Liga Letônia,Rapla,Valmiera Glass Via,2.56,Back Home
29453,19:00,Eua Ncaa,Illinois State,St. Louis Billikens,2.10,Back Home
